In [ ]:
import sys
sys.path.append('../Src')
import transformer_full
from libs import *

import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3,4,5,6,7,8,9,10,11'

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
# Sets all random seeds for the program (Python, NumPy, and TensorFlow)
keras.utils.set_random_seed(117)
tf.config.experimental.enable_op_determinism()

In [ ]:
wind_turbine = 'MF02'
model_name = 'pre_trained_1_v2'
project_folder = ".."
samples_per_day = 144
firts_training = True

In [ ]:
features_list = [
    wind_turbine + "_Gear Bearing Temp. Avg.",
    wind_turbine + "_Gear Bearing Temp.B Avg.",
    wind_turbine + "_Gear Bearing Temp.C Avg.",
    wind_turbine + "_Gear Oil Temp. Avg.",
    wind_turbine + "_Gear Oil Temp.Inlet Avg.",
    wind_turbine + "_Rotor RPM Max.",
    wind_turbine + "_Rotor RPM Avg."
]

In [ ]:
columns = ['gear_bearing_temp', 'gear_bearing_temp_b', 'gear_bearing_temp_c', 'gear_oil_temp', 'gear_oil_temp_inlet', 'rotor_rpm_max', 'rotor_rpm_avg']
data = pd.DataFrame(columns=columns)

In [ ]:
for year in [2019, 2020]:
    data_year = pd.read_pickle(f'{project_folder}/Data/MANF{year}.pkl')

    data_year.columns = data_year.columns.str.replace(r'\(\d+\)', '').str.strip()
    data_year = data_year[features_list]
    data_year.columns = columns

    data = pd.concat([data, data_year])

In [ ]:
data.info()

In [ ]:
data.reset_index(inplace=True)
data.rename(columns={'index': 'date'}, inplace=True)
data.head()

In [ ]:
data = data.sort_values(by='date').reset_index(drop=True)

# Data pre-processing
## Averaging duplicated values

In [ ]:
print(f'Number of duplicated dates: {data.shape[0] - data.date.nunique()}')

In [ ]:
# Averaging the values for each date duplicates
data = data.groupby('date').mean().reset_index()

In [ ]:
data[data.date.duplicated()]

## Filling missing values

In [ ]:
data[data['gear_bearing_temp'].isna()]

In [ ]:
data['gear_bearing_temp'].interpolate(method='pchip')

In [ ]:
for column in data.columns:
    if column != 'date':
        data[column].interpolate(method='pchip', inplace=True)

In [ ]:
data[data['gear_bearing_temp'].isna()]

## Dropping outliers

In [ ]:
# Smoothing the curve
for column in data.columns:
    if column != 'date':
        data[column] = data[column].rolling(samples_per_day*7).mean()
data = data.iloc[samples_per_day*7 - 1:].reset_index(drop=True)

In [ ]:
# Calculate z-scores for each column
data['z_scores'] = stats.zscore(data['gear_bearing_temp'], nan_policy='omit', axis=0)

# Define a threshold for outlier detection
threshold = 5

In [ ]:
data.loc[data['z_scores'].abs() > threshold]

In [ ]:
outliers = data.loc[data['z_scores'].abs() > threshold].shape
print(f'Number of outliers found: {outliers}')

In [ ]:
# Filtering out rows with z-scores greater than the threshold
data['gear_bearing_temp'] = np.where(data['z_scores'].abs() > threshold, np.nan, data['gear_bearing_temp'])
data = data.drop(columns=['z_scores'])

In [ ]:
# Filling again the NaN values assigned to the outliers
data['gear_bearing_temp'].interpolate(method='pchip', inplace=True)

## Feature engineering

In [ ]:
data['gear_oil_temp'] = np.mean([data['gear_oil_temp'], data['gear_oil_temp_inlet']], axis=0)
data.drop(columns=['gear_oil_temp_inlet', 'rotor_rpm_max'], inplace=True)

In [ ]:
fig, ax = plt.subplots(5, 1, figsize =(25,13))
sns.lineplot(data=data, x='date', y='gear_bearing_temp', ax=ax[0])
sns.lineplot(data=data, x='date', y='gear_bearing_temp_b', ax=ax[1])
sns.lineplot(data=data, x='date', y='gear_bearing_temp_c', ax=ax[2])
sns.lineplot(data=data, x='date', y='gear_oil_temp', ax=ax[3])
#sns.lineplot(data=data, x='date', y='gear_oil_temp_inlet', ax=ax[4])
#sns.lineplot(data=data, x='date', y='rotor_rpm_max', ax=ax[5])
sns.lineplot(data=data, x='date', y='rotor_rpm_avg', ax=ax[4])
_=plt.xticks(rotation=45)

## Reducing frequency

data.index = pd.to_datetime(data['date'])
data = pd.Series(data['gear_bearing_temp'])
data

data = data.resample(rule='H').mean()
data

data = pd.DataFrame(data).reset_index()

## Differencing by year to obtain stationarity

data['gear_bearing_temp'] = data['gear_bearing_temp'].diff(samples_per_day * 365)
data.dropna(inplace=True)

fig, ax = plt.subplots(figsize =(16,8))
sns.lineplot(data=data, x='date', y='gear_bearing_temp', ax=ax)
_=plt.xticks(rotation=45)

## Train-test Split

In [ ]:
scaler = MinMaxScaler()
scaler.fit(data.iloc[:,1:])

data.iloc[:,1:]=scaler.transform(data.iloc[:,1:])

In [ ]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        #end_ix = n_steps*i + n_steps
        end_ix = i + n_steps
        #start_ix = end_ix - n_steps
        # check if we are beyond the dataset
        if end_ix >= len(sequences):
            break
            # gather input and output parts of the pattern
        seq_x, seq_y = sequences.iloc[i:end_ix, 1:], sequences.iloc[end_ix, 1]
        #seq_x = list(sequences.iloc[i:end_ix, 1:].mean())
        #seq_y = sequences.iloc[end_ix, 1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
n_steps = samples_per_day
x_train, y_train = split_sequences(data, n_steps)
print(x_train.shape, y_train.shape)

# Model definition and training

In [ ]:
def get_compiled_model(x_train, y_train):
    input_shape = x_train.shape[1:]

    model = transformer_full.build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[100],
    mlp_dropout=0.3,
    dropout=0.2,
)

    model.compile(loss="mean_squared_error",
        optimizer = keras.optimizers.Adam(learning_rate=0.0003)
    )

    return model

In [ ]:
if firts_training:
    model = get_compiled_model(x_train, y_train)

else:
    # load the model and prepare for continuing training
    model = keras.models.load_model(f'{project_folder}/Models/{model_name}.h5')
    model.compile(loss="mean_squared_error",
        optimizer = keras.optimizers.Adam(learning_rate=0.0003)
    )

callbacks = [keras.callbacks.ModelCheckpoint(f"{project_folder}/Models/{model_name}.h5", save_best_only=True, monitor="val_loss"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=30, verbose=1),]

history = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    #validation_data=(x_val, y_val),
    shuffle=False,
    epochs=50,
    batch_size=64,
    callbacks=callbacks,
)

In [ ]:
model.save(f'{project_folder}/Models/{model_name}.h5')
history_df = pd.DataFrame(history.history)
history_df.to_csv(f'{project_folder}/Training_history/{model_name}_{wind_turbine}_history.csv', index=False)

loss = history.history['loss'][1:]
val_loss = history.history['val_loss'][1:]

In [ ]:
sns.set_theme(palette="ch:s=.25,rot=-.25")
fig,ax = plt.subplots(figsize=(8,8))
sns.lineplot(data=loss, ax = ax, color="b", label='Training Loss')
sns.lineplot(data=val_loss, ax = ax, color="r", label='Validation Loss')
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")

# save plot as image
plt.savefig(f'{project_folder}/Training_plots/pre_training/{model_name}_{wind_turbine}_loss.png')

In [ ]:
history_df.to_csv(f'{project_folder}/Training_history/{model_name}_history.csv', index=False)

# Model evaluation

In [ ]:
y_train_pred = model.predict(x_train)

In [ ]:
date_train = data.date
date_train = date_train[samples_per_day:]

In [ ]:
y_train = y_train.reshape(-1)
y_train_pred = y_train_pred.reshape(-1)

In [ ]:
date_train.shape

In [ ]:
train_results = pd.DataFrame({'date':date_train[2:],
                              'Real':y_train[:-2],
                              'Predicted': y_train_pred[2:]})
train_results

In [ ]:
biased_mean = np.mean(train_results['Real'] - train_results['Predicted'])
print("Biased mean: ", biased_mean)

train_results['Predicted'] += biased_mean

In [ ]:
train_results['Residual'] = abs(train_results.Real - train_results.Predicted)

In [ ]:
sns.set_theme(palette="ch:s=.25,rot=-.25")
fig,ax = plt.subplots(figsize=(16,8))
sns.lineplot(data=train_results, x='date', y='Real', ax = ax, color="b", label='Real')
sns.lineplot(data=train_results, x='date', y='Predicted', ax = ax, color="r", label='Predicted')

In [ ]:
window = samples_per_day * 7   #Averaged per week
#averaged_error = train_results.Residual.rolling(window = window).mean()[window-1:]
averaged_error = train_results.Residual.ewm(span=window).mean()

In [ ]:
mean = np.mean(averaged_error)
std = np.std(averaged_error)

In [ ]:
threshold_k3 = mean+3.*std
threshold_k6 = mean+6.*std
threshold_k7 = mean+7.*std

In [ ]:
sns.set_theme(palette="ch:s=.25,rot=-.25")
fig,ax = plt.subplots(figsize=(16,8))
sns.lineplot(x=train_results.date[window-1:], y=averaged_error, ax = ax, color="b", label='Error')

plt.axhline(y = threshold_k3, color = 'green', linestyle = '--', linewidth=2, label='mu + 3 * sigma')
plt.axhline(y = threshold_k6, color = 'fuchsia', linestyle = '--', linewidth=2, label='mu + 6 * sigma')
plt.axhline(y = threshold_k7, color = 'red', linestyle = '--', linewidth=2, label='mu + 7 * sigma')

plt.legend(frameon=False)

In [ ]:
# Saving the plot image
plt.savefig(f'{project_folder}/Training_plots/pre_training/{model_name}_{wind_turbine}_validation.png')